In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import cv2
from scipy.spatial import distance

In [2]:
mask_path = "../input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithMask"
no_mask_path = "../input/human-faces/Humans"

In [3]:
image_mask = []
target_mask = []
for i in os.listdir(mask_path):
    pic = os.path.join(mask_path + "/", i)
    image_mask.append(pic)
    target_mask.append(1)   

In [4]:
image_no_mask = []
target_no_mask = []
for i in os.listdir(no_mask_path):
    pic = os.path.join(no_mask_path + "/", i)
    image_no_mask.append(pic)
    target_no_mask.append(0)

In [5]:
mask = pd.DataFrame()
mask["image"] = image_mask
mask["target"] = target_mask

In [6]:
no_mask = pd.DataFrame()
no_mask["image"] = image_no_mask
no_mask["target"] = target_no_mask

In [7]:
data = pd.concat([mask, no_mask], axis=0, ignore_index=True)
data = shuffle(data)
data

,image,target
4497,../input/face-mask-12k-images-dataset/Face Mas...,1
12026,../input/human-faces/Humans/1 (1648).jpg,0
9452,../input/human-faces/Humans/1 (2821).jpg,0
5775,../input/human-faces/Humans/1 (5535).jpg,0
5108,../input/human-faces/Humans/1 (5716).jpg,0
...,...,...
5320,../input/human-faces/Humans/1 (886).jpg,0
4043,../input/face-mask-12k-images-dataset/Face Mas...,1
7025,../input/human-faces/Humans/1 (109).png,0
4070,../input/face-mask-12k-images-dataset/Face Mas...,1


In [8]:
import torch

from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

class dataset:
    def __init__(self, image_path, targets, resize = None, augmentation = None):
        
        self.image_path = image_path
        self.targets = targets
        self.resize = resize
        self.augmentation = augmentation
        
    def __len__(self):
        
        return len(self.image_path)

    def __getitem__(self, item):
        
        image = Image.open(self.image_path[item])
        
        image = image.convert("RGB")
        
        target = self.targets[item]
        
        if self.resize is not None:
            image = image.resize((
                self.resize[1], self.resize[0]
            ),
            resample = Image.BILINEAR
            )
            
        image = np.array(image)
        
        if self.augmentation is not None:
            augmented = self.augmentation(image=image)
            image = augmented["image"]
            
            
        image = np.transpose(image, (2,0,1)).astype(np.float32)
            
        return {
            "image" : torch.tensor(image, dtype= torch.float),
            "target" : torch.tensor(target, dtype= torch.long),
        }

In [16]:
#engine

import torch
import torch.nn as nn

def train(data_loader, model, optimizer, device):
    
    model.train()
    
    for data in data_loader:
        inputs = data["image"]
        targets = data["target"]
        
        inputs = inputs.to(device, dtype = torch.float)
        targets = targets.to(device, dtype= torch.float)
        
        optimizer.zero_grad()
        
        output = model(inputs)
       
        loss = nn.BCEWithLogitsLoss()(output, targets.view(-1,1))
        
        loss.backward()
        
        optimizer.step()
        
        
def evaluation(data_loader, model, device):
    
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in data_loader:
            
            inputs = data["image"]
            targets = data["target"]
            
            inputs = inputs.to(device, dtype= torch.float)
            targets = targets.to(device, dtype = torch.float)
            
            output = model(inputs)
            
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
            
    return final_outputs, final_targets

In [17]:
import pretrainedmodels

def get_model(pretrained):
    if pretrained:
        model = pretrainedmodels.__dict__["resnet18"](
                pretrained="imagenet"
        )
        
    else:
        model = pretrainedmodels.__dict__["resnet18"](
                pretrined = None
        )
        
    model.last_linear = nn.Sequential(
                nn.BatchNorm1d(512),
            nn.Dropout(p=0.25),
            nn.Linear(in_features = 512, out_features = 100),
            nn.ReLU(),
                nn.BatchNorm1d(100 ,eps=1e-05, momentum=0.1),
            nn.Dropout(p=0.5),
            nn.Linear(in_features = 100, out_features = 1 ),
        nn.Sigmoid()
       
            )
    return model

In [18]:


import albumentations
from sklearn import metrics
from sklearn.model_selection import train_test_split

device = 'cuda'
epoch = 10
targets = data.target.values
image = data.image.values
model = get_model(pretrained=True)
model.to(device)

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

aug = albumentations.Compose(
[
    albumentations.Normalize(
    mean, std, max_pixel_value=255.0, always_apply= True
    )
]
)


train_images, valid_images, train_targets, valid_targets = train_test_split(image, targets, stratify=targets, random_state=42)

train_dataset = dataset(image_path= train_images, targets = train_targets, resize = (100,100), augmentation=aug)

train_loader = torch.utils.data.DataLoader( train_dataset, batch_size = 16, shuffle = True, num_workers = 4)

valid_dataset = dataset(image_path= valid_images, targets= valid_targets, resize= (100,100), augmentation= aug)

valid_loader = torch.utils.data.DataLoader( valid_dataset, batch_size = 16, shuffle = True, num_workers = 4)

optimizer = torch.optim.Adam(model.parameters(), lr = 5e-4)






Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [19]:
epochs = 15
for epoch in range(epochs):
    
    train(train_loader, model, optimizer, device = device)
    
    prediction, valid_targets = evaluation( valid_loader, model, device = device )
    
    roc_auc = metrics.roc_auc_score(valid_targets, prediction)
    print(roc_auc)

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency e

0.9685631024930748


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


0.9827594459833794


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


...

0.948881108033241


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


0.9727290858725762


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


0.96019567867036


In [20]:
torch.save(model.state_dict(), './model_new')